In [131]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from os import walk
import xlwt 
from xlwt import Workbook 
import tensorflow as tf
from tensorflow import keras
from sklearn.cluster import KMeans

In [132]:
from keras.models import load_model
model = load_model("/content/drive/My Drive/project1/classifier2.h5")

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [133]:
model.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=["accuracy"])

In [134]:


def cluster_colors(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    wn_clusters = 3
    white_fg = False

    ### CHECK WHITE CLOTHES

    point1 = np.array(image[75,40,:])
    point2 = np.array(image[80,60,:])
    if ((point1[0]<245 and point1[0]>225) and (point1[1]<245 and point1[1]>225)  and (point1[2]<245 and point1[2]>225)) or ((point2[0]<245 and point2[0]>225) and (point2[1]<245 and point2[1]>225)  or (point2[2]<245 and point2[2]>225)):
        white_fg = True

    
    image = image.reshape((image.shape[0] * image.shape[1], 3))

    clt = KMeans(n_clusters = wn_clusters)
    clt.fit(image)

    centroids = clt.cluster_centers_

    ### HISTOGRAM

    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins = numLabels)
    hist = hist.astype("float")

    ### DELETE BACKGROUND

    index = 0
    n,m = centroids.shape

    for k,l in zip(range(0,n,1),centroids):
      for a,b in zip(range(0,n,1),centroids):
        if (k == a):
          continue
        else:
          if (l.sum()>b.sum()):
            centroids[[k,a]] = centroids[[a,k]]
            temp = hist[k]
            hist[k] = hist[a]
            hist[a] = temp
          else:
            centroids[[a,k]] = centroids[[k,a]]
            temp = hist[a]
            hist[a] = hist[k]
            hist[k] = temp

    if (white_fg == False):
      for counter in range(0,n,1):
        centroids_copy = np.array(centroids).reshape(centroids.shape)
        for i,j in zip(centroids_copy,range(0,n,1)):
          i = i.reshape((3,))
          if (i[0]<235 and i[0]>225) and (i[1]<235 and i[1]>225)  and (i[2]<235 and i[2]>225) and (i.sum()>=680 and i.sum()<=700 and i.min()>=225):
            np.delete(centroids,j,0)
            np.delete(hist,j,0)
            break
          

    bar = np.zeros((50, 300, 3), dtype = "uint8")
    startX = 0
    for (percent, color) in zip(hist, centroids):
            endX = startX + (percent * 300)
            cv2.rectangle(bar, (int(startX), 0), (int(endX), 50),color.astype("uint8").tolist(), -1)
            startX = endX

    n,m = centroids.shape
    if (n == 3):
      centroids = centroids[0:2,:]
      hist = hist[0:2]

    hist /= hist.sum()

    return (bar,centroids,hist)

In [135]:
def write_img(image,result,i):
  image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
  
  if result == 0:
    address = "/content/drive/My Drive/rtest3saves/gomlek/"+str(i)+".jpg"
  elif result == 1:
    address = "/content/drive/My Drive/rtest3saves/pantolon/"+str(i)+".jpg"
  elif result == 2:
    address = "/content/drive/My Drive/rtest3saves/t-shirt/"+str(i)+".jpg"
  cv2.imwrite(address,image)
  return address

In [136]:
wb = Workbook()
sheet = wb.add_sheet("Rapor",cell_overwrite_ok=True)

def sheet_write(result,centroid,i,address,hist):
  sheet.write(i+1,0,address)
  if result == 0:
    sheet.write(i+1,1, "gomlek")
  elif result == 1:
    sheet.write(i+1,1, "pantolon")
  elif result == 2:
    sheet.write(i+1,1, "t-shirt")
  (n,m) = centroid.shape
  sayac1=0
  for countit in range(2,(3*2+2),3):
    if (n==1 and sayac1==1):
      sheet.write(i+1,countit,"0")
      sheet.write(i+1,countit+1,"0")
      sheet.write(i+1,countit+2,"0")
    else:
      sheet.write(i+1,countit,str(centroid[sayac1,0]))
      sheet.write(i+1,countit+1,str(centroid[sayac1,1]))
      sheet.write(i+1,countit+2,str(centroid[sayac1,2]))
    sayac1 = sayac1 + 1
  
  sayac2=0
  for countit2 in range(3*2+2,7*2+2,4):
    if (n==1 and sayac2==1):
      sheet.write(i+1,countit2,"0")
      sheet.write(i+1,countit2+1,"0")
      sheet.write(i+1,countit2+2,"0")
      sheet.write(i+1,countit2+3,"0")
    else:
     cmyk_codes = list(rgb_to_cmyk(centroid[sayac2,:]/255))
     sheet.write(i+1,countit2,str(cmyk_codes[0]))
     sheet.write(i+1,countit2+1,str(cmyk_codes[1]))
     sheet.write(i+1,countit2+2,str(cmyk_codes[2]))
     sheet.write(i+1,countit2+3,str(cmyk_codes[3]))
    sayac2 = sayac2 + 1
  if (n==1):
    sheet.write(i+1,7*2+2,"1.0")
    sheet.write(i+1,7*2+3,"0.0")
  elif (n==2):
    sheet.write(i+1,7*2+2,str(hist[0]))
    sheet.write(i+1,7*2+3,str(hist[1]))
  sheet.write(i+1,7*2+4,str(n))

  



In [137]:
def rgb_to_cmyk(centro):
  k = 1 - centro.max()
  c = (1 - centro[0] - k) / (1 - k)
  m = (1 - centro[1] - k) / (1 - k)
  y = (1 - centro[2] - k) / (1 - k)
  return (c,m,y,k)


In [138]:
path_list = []
for (dirpath, dirnames, filenames) in walk("/content/drive/My Drive/rtest3"):
    path_list.extend(filenames)
    break
results = []
for i in range(0,20,1):
  path = dirpath + '/' + path_list[i]
  image = cv2.imread(path)
  bar,centroid,hist = cluster_colors(image)
  image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  image = cv2.resize(src=image,dsize=(125,100))
  image = image.reshape((1,100,125,3))
  result = model.predict_classes(image)
  image = image.reshape((100,125,3))
  address = write_img(image,result,i)
  sheet_write(result,centroid,i,address,hist)
  results.extend(result)

In [139]:
wb.save("/content/drive/My Drive/project1/rapor53.xls")